In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from shapely.geometry import Point
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras import regularizers
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
path = 'data/preprocessed'
file_name = 'combined_only'
df = pd.read_csv(f'../{path}/{file_name}.csv')
df.head(3)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon
0,1.252340e+12,1.325376e+09,0.0,0.0,0.0,153.0,52.458649,4.581200
1,1.252340e+12,1.325378e+09,0.0,0.0,0.0,153.0,52.458668,4.581167
2,1.252340e+12,1.325379e+09,0.0,0.0,0.0,153.0,52.458633,4.581183


In [3]:
df['mmsi'].value_counts().count(), df.shape

(354, (28581398, 8))

## Keep small sample

In [4]:
sample_size = 1000000 

start_index = 0  

# Create a smaller sequential subset
data = df.iloc[-sample_size:]

data.shape

(1000000, 8)

### Export small dataset

In [5]:
output_folder = '../data/preprocessed'
output_file = f'preproc_{int(sample_size/1000)}k.csv'

# Construct the full path
output_path = f'{output_folder}/{output_file}'

# Save the DataFrame to the specified path
data.to_csv(output_path, index=False)

# Recurrent Neural Network setup

## Data prep

In [6]:
number_vessel=data['mmsi'].value_counts().count()
print('Total number of vessels in our small dataset: ', number_vessel)

Total number of vessels in our small dataset:  21


### Split data by vessels

In [7]:
data.head()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon
27581398,1.337471e+14,1.457563e+09,11661.617188,38469.824219,8.0,333.200012,43.281300,16.269434
27581399,1.337471e+14,1.457564e+09,11401.474609,38274.378906,8.6,285.500000,43.293980,16.247747
27581400,1.337471e+14,1.457564e+09,12648.800781,39407.152344,8.5,314.200012,43.295353,16.228584
27581401,1.337471e+14,1.457565e+09,9486.599609,37535.726562,8.4,321.500000,43.317936,16.205826
27581402,1.337471e+14,1.457565e+09,8944.052734,37482.410156,8.4,320.700012,43.323536,16.198494


In [8]:
geometry = [Point(xy) for xy in zip(data['lon'], data['lat'])]
crs = {'init':'epsg:4326'}
geo = gpd.GeoDataFrame(data, #specify our data
                          crs=crs, #specify our coordinate reference system
                          geometry=geometry) #specify the geometry list we created
geo.head()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,geometry
27581398,1.337471e+14,1.457563e+09,11661.617188,38469.824219,8.0,333.200012,43.281300,16.269434,POINT (16.26943 43.28130)
27581399,1.337471e+14,1.457564e+09,11401.474609,38274.378906,8.6,285.500000,43.293980,16.247747,POINT (16.24775 43.29398)
27581400,1.337471e+14,1.457564e+09,12648.800781,39407.152344,8.5,314.200012,43.295353,16.228584,POINT (16.22858 43.29535)
27581401,1.337471e+14,1.457565e+09,9486.599609,37535.726562,8.4,321.500000,43.317936,16.205826,POINT (16.20583 43.31794)
27581402,1.337471e+14,1.457565e+09,8944.052734,37482.410156,8.4,320.700012,43.323536,16.198494,POINT (16.19849 43.32354)


In [9]:
X = geo.drop(columns = ['timestamp', 'lat', 'lon'], axis=1)
X.head(3)

,mmsi,distance_from_shore,distance_from_port,speed,course,geometry
27581398,1.337471e+14,11661.617188,38469.824219,8.0,333.200012,POINT (16.26943 43.28130)
27581399,1.337471e+14,11401.474609,38274.378906,8.6,285.500000,POINT (16.24775 43.29398)
27581400,1.337471e+14,12648.800781,39407.152344,8.5,314.200012,POINT (16.22858 43.29535)


In [10]:
# Extract coordinates from "Point" objects and convert to numeric data
coordinates = [(point.x, point.y) for point in geo['geometry']]
numeric_data = np.array(coordinates, dtype=np.float32)

# Convert numeric data to TensorFlow tensor
tensor = tf.convert_to_tensor(numeric_data)

print(tensor)

tf.Tensor(
[[16.269434 43.2813  ]
 [16.247747 43.29398 ]
 [16.228584 43.295353]
 ...
 [13.845977 43.44256 ]
 [13.807993 43.493008]
 [13.815951 43.414944]], shape=(1000000, 2), dtype=float32)


In [11]:
grouped = X.groupby('mmsi')

# List to store the NumPy arrays for each group
X_group_arrays = []
#y_group_arrays = []

# Iterate through each group and store the data as a NumPy array
for mmsi_value, group_df in grouped:
    # 'group_df' contains the subset of data for the current 'mmsi' group
    # Convert the relevant columns to a NumPy array and append it to the list
    X_group_array = group_df.drop(columns = ['mmsi', 'geometry'], axis=1).values
    X_group_arrays.append(X_group_array)
    
#    y_group_array = tensor
#    y_group_arrays.append(y_group_array)
    
assert(len(X_group_arrays) == number_vessel)
#assert(len(y_group_arrays) == number_vessel)


### Padding

In [12]:
X_pad = pad_sequences(X_group_arrays, dtype='float32', padding='post', value=-1000)
print('X_train_pad shape: ',X_pad.shape)

X_train_pad shape:  (21, 227003, 4)


In [13]:
pd.DataFrame(X_pad[0])

,0,1,2,3
0,11661.617188,38469.824219,8.0,333.200012
1,11401.474609,38274.378906,8.6,285.500000
2,12648.800781,39407.152344,8.5,314.200012
3,9486.599609,37535.726562,8.4,321.500000
4,8944.052734,37482.410156,8.4,320.700012
...,...,...,...,...
226998,-1000.000000,-1000.000000,-1000.0,-1000.000000
226999,-1000.000000,-1000.000000,-1000.0,-1000.000000
227000,-1000.000000,-1000.000000,-1000.0,-1000.000000
227001,-1000.000000,-1000.000000,-1000.0,-1000.000000


### Split train / test

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, tensor, test_size=0.3, shuffle=False)
print(X_train.shape)
print(X_test.shape)

ValueError: Found input variables with inconsistent numbers of samples: [21, 1000000]

In [ ]:
from tensorflow.keras.layers import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
from tensorflow.keras import layers

layer = Normalization()
layer.adapt(X_train)

model = Sequential()
model.add(layer)
model.add(layers.SimpleRNN(20, activation='tanh', input_shape=(227003, 4)))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='linear'))
model.summary()

## init_model

In [ ]:
# def init_model(input_shape):
    

    # 1 - RNN architecture
    # ======================    
    # model = models.Sequential()    
    # # Normalizing Inputs
    # # model.add(normalizer)
    
    # # Recurrent Layer
    # model.add(layers.LSTM(units=64, activation='tanh', return_sequences=False, 
    #                       recurrent_dropout=0.5, dropout=0.5, input_shape=input_shape))
    
    # # Hidden Dense Layer that we are regularizing
    # # reg_l2 = regularizers.L2(0.5)
    # # model.add(layers.Dense(32, activation="relu", kernel_regularizer = reg_l2))
    
    # model.add(layers.Dense(32, activation="relu"))
    # model.add(layers.Dropout(rate=0.5))
    
    # # Predictive Dense Layer
    # ### model.add(layers.Dense(1, activation='linear'))  
    
    # model.add(layers.Dense(1, activation='sigmoid'))
    
    # # model.build(input_shape=(None, input_shape)) 
 

    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

    # return model

In [ ]:
def init_model(X_train):
    model = models.Sequential() 
    
    model.add(
        layers.Masking(
            mask_value=-1000, 
            input_shape=(X_train.shape[1], X_train.shape[2])
                       )
        )
    
    model.add(BatchNormalization()) 
    
    model.add(layers.LSTM(64, activation='tanh'))
    
    model.add(layers.Dense(2, activation='sigmoid'))
    
    initial_learning_rate = 0.1
    optimizer = optimizers.Adam(learning_rate=initial_learning_rate)
    
    model.compile(loss='mse', optimizer=optimizer, metrics=[metrics.Precision(), metrics.Accuracy()])
    
    return model

# Model test and tuning

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2])
input_shape

In [ ]:
model = init_model(X_train)
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


es = EarlyStopping(monitor="val_loss",
                    patience=4,
                    mode="min",
                    restore_best_weights=True)

# Create ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",   # Monitor validation loss
    factor=0.5,            # Factor by which the learning rate will be reduced (new_lr = lr * factor)
    patience=2,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6,           # Lower bound on the learning rate
    verbose=1
)

history = model.fit(
    X_train,
    y_train_array,
    validation_split=0.3,
    shuffle=False,
    batch_size=24,
    epochs=20,
    callbacks=[es], #, reduce_lr],   # Add both callbacks to the list
    verbose=1
)

## Plot history

In [ ]:
def plot_history(history, metric):
    
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # --- LOSS: binary_crossentropy --- 
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('binary_crossentropy')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Validation'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- METRICS:accuracy ---
    
    ax[1].plot(history.history[metric])
    ax[1].plot(history.history['val_'+metric])
    ax[1].set_title(metric)
    ax[1].set_ylabel(metric)
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Validation'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
                        
    return ax

In [ ]:
history.history.keys()

In [ ]:
plot_history(history, 'precision')